# Find Characters and Predict Label from given Image

In [1]:
import cv2 as cv
import pickle as pk
import numpy as np
with open('let_KNN_clf.pickle','rb') as f:
    let_KNN_clf = pk.load(f)
with open('dig_KNN_clf.pickle','rb') as f:
    dig_KNN_clf = pk.load(f)
with open('dig_MLP_clf.pickle','rb') as f:
    dig_MLP_clf = pk.load(f)
with open('let_MLP_clf.pickle', 'rb') as f:
    let_MLP_clf = pk.load(f)
with open('bal_MLP_clf.pickle','rb') as f:
    bal_MLP_clf = pk.load(f)
with open('dig_MLP_clf_hog.pickle', 'rb') as f:
    dig_MLP_clf_hog = pk.load(f)


In [2]:
def get_roi_all(image): 
    ''' find objects in image, return vector of segmented images and respective rectangles'''
    # find contours
    image = cv.GaussianBlur(image,(5,5),1)
    ret, image = cv.threshold(image, 90, 255, cv.THRESH_BINARY_INV)
    ctrs, hier = cv.findContours(image.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # find rectangles
    rects = [cv.boundingRect(ctr) for ctr in ctrs]
    #copy each object to image vector
    #rect = [x, y, width, height]
    im_vec = [image[rect[1]-int(rect[3]*0.2):rect[1]+int(rect[3]*1.2), 
                    rect[0]-int(rect[2]*0.2):rect[0]+int(rect[2]*1.2)] for rect in rects]
    return im_vec, rects

def draw_named_rect(image, rectangle, text):
    '''draw rectangle and name it'''
    rect = rectangle
    img = image
    cv.rectangle(img, (rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]), 0, 1)
    cv.putText(img, text,(rect[0],rect[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.9,0, 1)
    return img
      
def adjust_img(image, width, height):
    '''resize to (height, width)'''
    image = cv.resize(image, dsize = (height,width),interpolation=cv.INTER_AREA)
    image_vec = image.reshape(1,-1)
    return image_vec

def pred_character(classifier, im_vec, target):
    '''classifies object and returns ascii code'''
    #dictionaries to look up class and character depending on target
    balanced_dict = dict(zip([i for i in range(0,47)],
                        [str(i) for i in range(0,10)]+['A','B','C','D','E','F','G',
                                                      'H','I','J','K','L','M','N','O',
                                                      'P','Q','R','S','T','U','V','W',
                                                      'X','Y','Z']+['a','b','d','e','f',
                                                        'g','h','n','q','r','t']))
    letters_dict = dict(zip([i for i in range(1,27)],
                           [chr(n) for n in range(65,91)]))
    # Prediction
    pred  = classifier.predict(np.array(im_vec, 'float64'))
    # set character depending on target_type
    if target== 'digits' or target == 'all':
        pred_char = balanced_dict[pred[0]]
    elif target == 'letters':
        #classes from 1 to 26 (upper + lower case in one class)
        #offset = 64 # A --> 65
        pred_char = letters_dict[pred[0]]
    else:
        raise ValueError('target must be "digits", "all" or "letters"')
    return pred, pred_char

def start_recognition(image_name, clf, target_type):
    '''predict character and draw label on image
    '''
    #read image
    im = cv.imread(image_name,cv.IMREAD_GRAYSCALE)
    # find the character in image
    images_roi, rects = get_roi_all(im)
    for idx, rect in enumerate(rects):
        # fit the image to data set
        try:
            im_vec = adjust_img(images_roi[idx], 28 ,28)
        except Exception:
            #padding in function get_roi_all() might lead to error
            continue
        #predict and return ascii code
        pred, pred_char = pred_character(clf, im_vec, target_type)
        #draw classification
        im = draw_named_rect(im, rect, pred_char)
    return im


In [ ]:
im_in = 'letters_digits.jpg'
im_out = start_recognition(im_in, bal_MLP_clf, 'all')
cv.imshow('',im_out)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('letters_digits_out_bal.jpg', im_out)